In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2

# Shapes

(Shapes is an abbreviation to the more complete BAShapes dataset)

In [2]:
from metrics.load_expl import nc_load_graphs

In [25]:
dataset = "Infection"
model_architecture = "GCN"
expl = "rgexpl"
MODE = "train"

gnn_num_layers = { #based on the hyper-params of networks
    "GCN": 3,
    "GraphSAGE": 2,
    "GIN": 3,
    "Cheb": 2
}

graphs = nc_load_graphs(DATASET=dataset,
                 MODEL=model_architecture,
                 EXPL=expl,
                 MODE=MODE,
                 verbose=True,
                 lamb=0.001,
                 raw=False,
                 normalize=True)

0.996 ,1.000 ,1.000 ,1.000 


In [ ]:
# G = graphs[1][586]

# color_map = []
# for node in G:
#     if nx.get_node_attributes(G, "node_imp")[node] > 0:
#         color_map.append('red')
#     else: 
#         color_map.append('blue')      

# plt.figure(figsize=(15,15))
# nx.draw(G, node_size=30, with_labels=True, node_color=color_map)

# nans = np.isnan(list(nx.get_node_attributes(G,"node_imp").values()))
# print(np.where(nans)[0])
# print(len(G.edges()))

Note that each graph contains attribution scores for all nodes in the graph (for convenience). Typically, nodes outside the n-hop neighborhood will have 0 score

In [ ]:
# g = graphs[0][0]

# print(nx.get_node_attributes(g,"node_imp").keys())
# print(nx.get_node_attributes(g,"node_imp").values())
# print(nx.get_node_attributes(g,"node_imp_norm").values())
# print(np.any(np.array(list(nx.get_node_attributes(g,"node_imp").values())) == np.nan))
# plt.hist((nx.get_node_attributes(g,"node_imp").values()));

### plausibility on SHAPES

In [3]:
from metrics.plausibility import get_shapes_plausibility

In [ ]:
dataset = "Infection"
model_architecture = "GCN"
expl = "rgexpl"
MODE = "train"

graphs = nc_load_graphs(DATASET=dataset,
                 MODEL=model_architecture,
                 EXPL=expl,
                 MODE=MODE,
                 verbose=True,
                 lamb=0.001,
                 raw=False,
                 normalize=True)

plausibility_per_class = get_shapes_plausibility(graphs, gnn_num_layers[model_architecture]) 
print(plausibility_per_class)

Results for RGExplainer
GIN: [nan, 0.622484504512572, 0.8944852292768959, 0.9747826086956521]
GCN: [nan, nan, nan, nan]
Cheb: [nan, 0.9754359217816235, 0.9942622950819672, 0.9857142857142858]
SAGE: [nan, 0.7863072064942769, 0.8179687500000001, 0.70625]

### Sufficiency

In [27]:
from metrics.NC_F1_fidelity import build_expl , compute_fidelity
from models.models_BAShapes import GraphSAGE_framework as framework
from torch_geometric.datasets import BAShapes

In [28]:
gcn , graphs = build_expl(dataset,model_architecture,BAShapes,framework,expl=expl)
        
for c in range(len(graphs.keys())):
    if not graphs[c] == None:
        suf = compute_fidelity(gcn, graphs, y=c)
    else:
        suf = float("nan")

    print("Suff. class {} = {:.3f}".format(c, suf))

Suff. class 0 = 0.909
Suff. class 1 = 0.675
Suff. class 2 = -0.008
Suff. class 3 = 0.874


In [17]:
gcn,graphs = build_expl(datset, model, dataset_fun,framework,expl=expl,MODE=MODE)

suf1, comp1 = compute_fidelity(gcn,graphs,y=1,color=False)
suf0, comp0 = compute_fidelity(gcn,graphs,y=0,color=False)

NameError: name 'DATASET' is not defined

In [46]:
f1_1 = 2*(((1-suf1) * (comp1))/((1-suf1) + (comp1)))
f1_0 = 2*(((1-suf0) * (comp0))/((1-suf0) + (comp0)))

print("f1 class 0 :", f1_0)
print("f1 class 1 :", f1_1)

f1 class 0 : 0.34425428130625607
f1 class 1 : 0.7225875146072238
